In [2]:
from xlsxwriter.utility import xl_rowcol_to_cell
from datetime import datetime, date
from pandas import DataFrame as df
from pathlib import Path
import pandas as pd
from os import path
import datetime
import time
import os

pd.options.display.float_format = "{:,.2f}".format

from mthly_inforce_file import InforceFile

%load_ext autoreload
%autoreload 2

In [2]:
inforce_fldr = r'C:\Users\S0053071\Repos\Orion_Process_Backup\Inforce_Archive'

In [92]:
for file_name in os.listdir(inforce_fldr):
    print(file_name)

01_2025_IUL_Fund_Values_RGA.txt
07_2024_IUL_Fund_Values_RGA.txt
08_2024_IUL_Fund_Values_RGA.txt
09_2024_IUL_Fund_Values_RGA.txt
10_2024_IUL_Fund_Values_RGA.txt
11_2024_IUL_Fund_Values_RGA.txt
12_2024_IUL_Fund_Values_RGA.txt


In [3]:
def process_subdir_hdgfiles(target_dir, files_to_process=None):
    """
    Iterates over the subdirectories of a given directory and prints their paths.

    Args:
        target_dir: The path to the directory to iterate over.
    """
    
    final_cols = ['HedgeFile_Dt', 'Attrib_Type', 'HedgeDate', 'CompID', 'Indicator', 'Bbg_Idx', 'Fund_Name', 'Opt_Type', 'Strike', 'Cap/Rate', 'Budget', 'PolicyCount', 'Base_Liab_Ntnl', 'Notional']
    df_cols = ['HedgeDate',	'CompID', 'Indicator', 'Bbg_Idx', 'Fund_Name', 'Opt_Type', 'Strike', 'Cap/Rate', 'Budget', 'PolicyCount', 'Base_Liab_Ntnl', 'Notional']


    # orig_file = 'iul_liab_summary.csv'
    # true_file = 'iul_liab_summary_TRUE_UP.csv'
    inforce_dict = {}
    
    target_dir = Path(target_dir)

    if files_to_process is None:
        # Set directories to all the valid directories
        # directories_to_process = [x[1] for x  in os.walk(target_dir)]
        # directories_to_process = directories_to_process[0]
        files_to_process = [f.name for f in target_dir.iterdir() if f.is_file()]
    
    for dirpath, _, filenames in os.walk(target_dir):
        
        for fname in filenames:

            if fname in files_to_process:

                inf_file_path = os.path.join(dirpath, fname)
                inf_file = InforceFile(inf_file_path)
                inforce_dict[inf_file.hedge_date] = inf_file
                    
    return inforce_dict

In [ ]:
inf_dict = process_subdir_hdgfiles(inforce_fldr)

Reading in Helper Dataframes (used for static lookups to obtain values for related fields)
REMINDER TO UPDATE NEW BUDGET INFO in Static_Assumptions Folder for ProductDetailsByHedgeDate.csv!!!
Reading Data from \\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\Static_Assumptions\HedgeDates.csv
Reading Data from \\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\Static_Assumptions\HdgFctrLU.csv
Reading Data from \\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\Static_Assumptions\CoPlanInd_to_Prod.csv
Reading Data from \\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\Static_Assumptions\FundID_to_Indicator.csv
Reading Data from \\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\Static_Assumptions\ProductDetailsByHedgeDate.csv
Reading Data from \\rga

In [6]:
print(list(inf_dict.keys())[0])
jan_inf =  inf_dict[list(inf_dict.keys())[0]].inforce_df

2025-01-02


In [9]:
jan_inf.columns

Index(['InforceDt', 'AsOfDt', 'CompID', 'PolicyNum', 'PHA_Num', 'Plan',
       'Indicator', 'Base_Liab_Ntnl', 'Idx_Credit', 'HedgeDate', 'ExpiryDt',
       'Product_Detail', 'Product', 'Budget', 'Part', 'Cap', 'Floor',
       'Spec_Rate', 'Spread', 'Asset_Charge', 'Multiplier', 'HdgFctr',
       'Strike', 'Adj_Liab_Ntnl'],
      dtype='object')

In [ ]:
inf_output_cols = ['HedgeDate', 'CompID', 'PolicyNum', 'AsOfDt', 'PHA_Num', 'Plan',
       'Indicator', 'Part', 'Cap', 'Floor', 'Spec_Rate', 'Spread', 'Asset_Charge', 
       'Multiplier', 'Strike', 'Budget', 'HedgeRatio', 'HdgFctr', 'Base_Liab_Ntnl', 
       'Adj_Liab_Ntnl']

In [7]:
jan_inf

,InforceDt,AsOfDt,CompID,PolicyNum,PHA_Num,Plan,Indicator,Base_Liab_Ntnl,Idx_Credit,HedgeDate,...,Part,Cap,Floor,Spec_Rate,Spread,Asset_Charge,Multiplier,HdgFctr,Strike,Adj_Liab_Ntnl
1,2024-01-01,2024-12-23,1,UE000006,84,1U145500,INX150,230.15,NaN,2024-01-02,...,1.00,0.09,0.01,0.00,0.00,0.00,0.00,0.97,1.01,223.25
2,2024-02-01,2024-12-23,1,UE000006,85,1U145500,INX150,235.35,NaN,2024-02-01,...,1.00,0.09,0.01,0.00,0.00,0.00,0.00,0.97,1.01,228.29
3,2024-03-01,2024-12-23,1,UE000006,86,1U145500,INX150,273.14,NaN,2024-03-01,...,1.00,0.09,0.01,0.00,0.00,0.00,0.00,0.97,1.01,264.95
4,2024-04-01,2024-12-23,1,UE000006,87,1U145500,INX150,272.31,NaN,2024-04-01,...,1.00,0.09,0.01,0.00,0.00,0.00,0.00,0.97,1.01,264.14
5,2024-05-01,2024-12-23,1,UE000006,88,1U145500,INX150,260.97,NaN,2024-05-01,...,1.00,0.09,0.01,0.00,0.00,0.00,0.00,0.97,1.01,253.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1249564,2024-12-01,2024-12-20,1,U0982402,1,1U147800,INXLOM,1.22,NaN,2024-12-02,...,1.00,0.12,0.00,0.00,0.00,0.02,0.24,0.95,1.00,1.44
1249565,2024-12-01,2024-12-20,1,U0982402,1,1U147800,INXHIM,1.19,NaN,2024-12-02,...,1.00,0.12,0.00,0.00,0.00,0.04,0.60,0.95,1.00,1.81
1249566,2024-12-01,2024-12-20,1,U0982402,1,1U147800,INDEX,1.24,NaN,2024-12-02,...,1.00,0.10,0.00,0.00,0.00,0.00,0.00,0.95,1.00,1.18
1249567,2024-12-01,2024-12-20,1,U0982402,1,1U147800,MARPRT,1.24,NaN,2024-12-02,...,2.40,10.00,0.00,0.00,0.00,0.00,0.00,0.95,1.00,2.83


In [101]:
inf_summary_view = None

for hdg_dt, inf_obj in inf_dict.items():

    inf_summary_df = inf_obj.inforce_df.groupby('HedgeDate', as_index=False).agg({'PolicyNum':'count', 'Base_Liab_Ntnl': 'sum', 'Adj_Liab_Ntnl': 'sum'})
    inf_summary_df['Inforce_AsOfDt'] = hdg_dt

    if inf_summary_view is None:
        inf_summary_view = inf_summary_df
    else:
        inf_summary_view = pd.concat([inf_summary_view, inf_summary_df]).reset_index(drop=True)


cols_ordered = ['Inforce_AsOfDt', 'HedgeDate', 'PolicyNum', 'Base_Liab_Ntnl', 'Adj_Liab_Ntnl']
inf_summary_view = inf_summary_view[cols_ordered]
inf_summary_view.rename(columns={'PolicyNum': 'PolicyCount'}, inplace=True)


In [7]:
inforce_idx_credits_view = None

for hdg_dt, inf_obj in inf_dict.items():

    idx_credit_eff_dt = inf_obj.raw_inforce_df['HedgeDate'].min()
    credit_df = inf_obj.raw_inforce_df
    credit_df = credit_df[credit_df['HedgeDate']==idx_credit_eff_dt]
    credit_df = credit_df.groupby(['HedgeDate', 'CompID', 'Indicator'], as_index=False).agg({'PolicyNum':'count', 'Base_Liab_Ntnl': 'sum', 'Adj_Liab_Ntnl': 'sum', 'Idx_Credit': 'sum'})
    credit_df['Inforce_AsOfDt'] = hdg_dt

    if inforce_idx_credits_view is None:
        inforce_idx_credits_view = credit_df
    else:
        inforce_idx_credits_view = pd.concat([inforce_idx_credits_view, credit_df]).reset_index(drop=True)


cols_ordered = ['Inforce_AsOfDt', 'HedgeDate', 'CompID', 'Indicator', 'PolicyNum', 'Base_Liab_Ntnl', 'Adj_Liab_Ntnl', 'Idx_Credit']
inforce_idx_credits_view = inforce_idx_credits_view[cols_ordered]
inforce_idx_credits_view.rename(columns={'PolicyNum': 'PolicyCount'}, inplace=True)

In [8]:
inforce_idx_credits_view

,Inforce_AsOfDt,HedgeDate,CompID,Indicator,PolicyCount,Base_Liab_Ntnl,Adj_Liab_Ntnl,Idx_Credit
0,2025-01-02,2023-12-01,1,INDEX,50933,0.00,0.00,"3,424,006.21"
1,2025-01-02,2023-12-01,1,INX150,2850,0.00,0.00,"578,880.35"
2,2025-01-02,2023-12-01,1,INXFEE,20412,0.00,0.00,"1,644,949.24"
3,2025-01-02,2023-12-01,1,INXHIM,5565,0.00,0.00,"400,052.18"
4,2025-01-02,2023-12-01,1,INXLOM,2296,0.00,0.00,"79,225.98"
...,...,...,...,...,...,...,...,...
79,2024-12-02,2023-11-01,1,NASIDX,2021,0.00,0.00,"64,381.92"
80,2024-12-02,2023-11-01,26,INDEX,1030,0.00,0.00,"48,395.38"
81,2024-12-02,2023-11-01,26,INX150,225,0.00,0.00,"3,661.52"
82,2024-12-02,2023-11-01,26,INXFEE,198,0.00,0.00,"15,219.68"


In [10]:
inforce_idx_credits_view.to_clipboard(index=False)

In [23]:

inf_summary_view

,Inforce_AsOfDt,HedgeDate,PolicyCount,Base_Liab_Ntnl,Adj_Liab_Ntnl
0,2025-01-02,2024-01-02,91481,"58,555,665.68","60,269,646.10"
1,2025-01-02,2024-02-01,92613,"59,738,832.95","61,046,020.25"
2,2025-01-02,2024-03-01,93537,"55,548,864.97","57,006,464.75"
3,2025-01-02,2024-04-01,94712,"55,925,079.27","57,707,624.57"
4,2025-01-02,2024-05-01,95533,"52,870,910.45","56,815,800.07"
...,...,...,...,...,...
77,2024-12-02,2024-07-01,97677,"63,052,655.94","66,348,904.94"
78,2024-12-02,2024-08-01,98210,"58,957,980.57","61,640,511.06"
79,2024-12-02,2024-09-03,99004,"60,257,097.85","61,307,500.49"
80,2024-12-02,2024-10-01,99617,"69,745,247.97","70,717,961.81"


In [25]:
inf_summary_view.to_clipboard()

# Note: Saved results of to_clipboard() here:
save_file = r'C:\Users\S0053071\Repos\Orion_Process_Backup\Scalar_Calcs\Inforce_Ntnl_OverTime.xlsx'


In [ ]:
list(inf_dict.keys())[0]

In [12]:
inf_dict[list(inf_dict.keys())[0]].inforce_df['ExpiryDt'].unique()

array([datetime.date(2025, 1, 2), datetime.date(2025, 1, 31),
       datetime.date(2025, 2, 28), datetime.date(2025, 4, 1),
       datetime.date(2025, 5, 1), datetime.date(2025, 6, 3),
       datetime.date(2025, 7, 1), datetime.date(2025, 8, 1),
       datetime.date(2025, 9, 3), datetime.date(2025, 10, 1),
       datetime.date(2025, 10, 31), datetime.date(2025, 12, 2)],
      dtype=object)

In [27]:
test_df = inf_dict[list(inf_dict.keys())[0]].inforce_df
test_raw_df = inf_dict[list(inf_dict.keys())[0]].raw_inforce_df

In [15]:
test_df[test_df['ExpiryDt']==date(2025, 1, 2)]

,InforceDt,AsOfDt,CompID,PolicyNum,PHA_Num,Plan,Indicator,Base_Liab_Ntnl,Idx_Credit,HedgeDate,...,Part,Cap,Floor,Spec_Rate,Spread,Asset_Charge,Multiplier,HdgFctr,Strike,Adj_Liab_Ntnl
1,2024-01-01,2024-12-23,1,UE000006,84,1U145500,INX150,230.15,NaN,2024-01-02,...,1.00,0.09,0.01,0.00,0.00,0.00,0.00,0.97,1.01,223.25
14,2024-01-01,2024-12-23,1,UE000006,84,1U145500,INXFEE,271.27,NaN,2024-01-02,...,1.00,10.00,0.00,0.00,0.08,0.00,0.00,0.97,1.08,263.13
27,2024-01-01,2024-12-09,1,UE000011,86,1U145500,INX150,292.83,NaN,2024-01-02,...,1.00,0.09,0.01,0.00,0.00,0.00,0.00,0.97,1.01,284.05
40,2024-01-01,2024-12-09,1,UE000012,84,1U145500,INX150,234.17,NaN,2024-01-02,...,1.00,0.09,0.01,0.00,0.00,0.00,0.00,0.97,1.01,227.14
53,2024-01-01,2024-12-20,1,UE000013,77,1U145500,INX150,61.94,NaN,2024-01-02,...,1.00,0.09,0.01,0.00,0.00,0.00,0.00,0.97,1.01,60.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248686,2024-01-01,2024-12-28,1,U0982048,1,1U147800,INDEX,"1,093.09",NaN,2024-01-02,...,1.00,0.10,0.00,0.00,0.00,0.00,0.00,0.97,1.00,"1,060.30"
1249189,2024-01-01,2024-12-15,1,U0982129,1,1U147800,INDEX,495.43,NaN,2024-01-02,...,1.00,0.10,0.00,0.00,0.00,0.00,0.00,0.97,1.00,480.57
1249190,2024-01-01,2024-12-15,1,U0982129,1,1U147800,NASIDX,495.43,NaN,2024-01-02,...,1.00,0.10,0.00,0.00,0.00,0.00,0.00,0.97,1.00,480.57
1249191,2024-01-01,2024-12-15,1,U0982130,1,1U147800,INDEX,494.90,NaN,2024-01-02,...,1.00,0.10,0.00,0.00,0.00,0.00,0.00,0.97,1.00,480.05


In [16]:
test_df['Idx_Credit'].sum()

np.float64(-0.33)

In [18]:
test_df.groupby('HedgeDate', as_index=False).agg({'PolicyNum':'count', 'Base_Liab_Ntnl': 'sum', 'Adj_Liab_Ntnl': 'sum'})

,HedgeDate,PolicyNum,Base_Liab_Ntnl,Adj_Liab_Ntnl
0,2024-01-02,91481,"58,555,665.68","60,269,646.10"
1,2024-02-01,92613,"59,738,832.95","61,046,020.25"
2,2024-03-01,93537,"55,548,864.97","57,006,464.75"
3,2024-04-01,94712,"55,925,079.27","57,707,624.57"
4,2024-05-01,95533,"52,870,910.45","56,815,800.07"
5,2024-06-03,96530,"52,527,852.93","54,702,411.38"
6,2024-07-01,97451,"62,875,785.40","66,162,550.82"
7,2024-08-01,97982,"58,775,614.71","61,446,256.05"
8,2024-09-03,98771,"59,036,732.77","60,137,006.81"
9,2024-10-01,99381,"69,500,233.99","70,467,046.85"


In [26]:
test_df.columns

Index(['InforceDt', 'AsOfDt', 'CompID', 'PolicyNum', 'PHA_Num', 'Plan',
       'Indicator', 'Base_Liab_Ntnl', 'Idx_Credit', 'HedgeDate', 'ExpiryDt',
       'Product_Detail', 'Product', 'Budget', 'Part', 'Cap', 'Floor',
       'Spec_Rate', 'Spread', 'Asset_Charge', 'Multiplier', 'HdgFctr',
       'Strike', 'Adj_Liab_Ntnl'],
      dtype='object')

In [45]:
idx_credit_dt = test_raw_df['HedgeDate'].min()

In [46]:
test_raw_df[(test_raw_df['Idx_Credit'].notna()) & (test_raw_df['HedgeDate']==idx_credit_dt) ].groupby(['HedgeDate', 'Indicator'], as_index=False).agg({'PolicyNum':'count', 'Idx_Credit': 'sum'})

,HedgeDate,Indicator,PolicyNum,Idx_Credit
0,2023-12-01,INDEX,51298,"3,455,428.03"
1,2023-12-01,INX150,2983,"581,564.69"
2,2023-12-01,INXFEE,20209,"1,656,407.39"
3,2023-12-01,INXHIM,5548,"400,052.18"
4,2023-12-01,INXLOM,2277,"79,225.98"
5,2023-12-01,INXSPC,1193,"21,086.53"
6,2023-12-01,MARPRT,3430,"254,464.29"
7,2023-12-01,NASIDX,2127,"78,309.06"


datetime.date(2023, 12, 1)

In [35]:
test_raw_df[(test_raw_df['Idx_Credit'].notna()) & (test_raw_df['HedgeDate']==idx_credit_dt) ].groupby(['HedgeDate'], as_index=False).agg({'PolicyNum':'count', 'Idx_Credit': 'sum'})

,HedgeDate,PolicyNum,Idx_Credit
0,2023-12-01,89065,"6,526,538.15"


In [47]:
idx_credit_df = test_raw_df[(test_raw_df['Idx_Credit'].notna()) & (test_raw_df['HedgeDate']==idx_credit_dt) ].groupby(['HedgeDate', 'Indicator'], as_index=False).agg({'PolicyNum':'count', 'Idx_Credit': 'sum'})

In [90]:
def get_xl_hdr_rng(df: pd.DataFrame):
    return xl_rowcol_to_cell(0,0) + ':' + xl_rowcol_to_cell(0, len(df.columns)-1)

def get_xl_col_rng(df: pd.DataFrame, col_idx: int):
    return xl_rowcol_to_cell(1, col_idx) + ':' + xl_rowcol_to_cell(0, len(df.columns)-1)

def get_xl_rng_for_df(df: pd.DataFrame):
    rows, columns = df.shape
    return xl_rowcol_to_cell(0,0) + ':' + xl_rowcol_to_cell(rows, columns-1)


In [91]:
get_xl_rng_for_df(inf_summary_view)

'A1:E83'

In [85]:
test_xl_file = r'C:\Users\S0053071\Repos\Orion_Process_Backup\TEST\test_xl_file.xlsx'

# writer = pd.ExcelWriter(test_xl_file, engine='xlsxwriter')

eng_kwargs = {'options': {'strings_to_numbers': True}}

# if_sheet_exists='replace', 

# with pd.ExcelWriter(test_xl_file, engine='xlsxwriter', date_format="yyyy/mm/dd", engine_kwargs=eng_kwargs) as writer:
with pd.ExcelWriter(test_xl_file, engine='xlsxwriter', engine_kwargs=eng_kwargs) as writer:
# with pd.ExcelWriter(test_xl_file, engine='openpyxl', mode='a', engine_kwargs=eng_kwargs) as writer:
# with pd.ExcelWriter(test_xl_file, engine='openpyxl', mode='a') as writer:

    idx_credit_df_sheet = 'Idx_Credits'
    inf_summary_view_sheet = 'Inforce_Summary'
    
    # Get Workbook
    xl_wb =  writer.book
    
    # Add a header format.
    header_format = xl_wb.add_format(
    {
        "border": 0,
        "bold": True,
        "text_wrap": False,
        # "valign": "top",
        "align": "center",
        "bg_color": "#D9D9D9",
        "font_color": "#0070C0",
        # "border": 1,
    })

    # Add a few formats for convenience
    # border_fmt = xl_wb.add_format({"align": "center"})
    date_fmt = xl_wb.add_format({"num_format": "yyyy/mm/dd", "align": "center"})
    non_numeric_fmt = xl_wb.add_format({"align": "center"})
    pct_fmt = xl_wb.add_format({"num_format": "0.00%", "align": "center"})
    num_fmt = xl_wb.add_format({"num_format": "#,##0", "align": "center"})


    dt_flds = ['HedgeDate', 'AsOfDt', 'ExpiryDt', 'InforceDt', 'Inforce_AsOfDt']
    pct_flds = ['Part', 'Cap', 'Cap/Rate', 'Floor',	'Spec_Rate', 'Spread', 'Asset_Charge', 'Multiplier', 'Strike', 'Budget', 'HdgFctr', 'HedgeAssetPct']
    numeric_flds = ['Idx_Credit', 'PolicyNum', 'PolicyCnt', 'PolicyCount', 'Base_Liab_Ntnl', 'Adj_Liab_Ntnl', 'Notional', 'Ntnl']

    def get_fmt_type(col_name):
        if col_name in pct_flds:
            return pct_fmt
        elif col_name in numeric_flds:
            return num_fmt
        elif col_name in dt_flds:
            return date_fmt
        else:
            return non_numeric_fmt
        
    idx_credit_df.to_excel(writer, index=False, header=True, sheet_name=idx_credit_df_sheet, startrow=0 , startcol=0)
    inf_summary_view.to_excel(writer, index=False, header=True, sheet_name=inf_summary_view_sheet, startrow=0 , startcol=0)

    def write_df_to_sheet_w_fmt(df: pd.DataFrame, sheet_name):

        # Create the sheet and write to it
        df.to_excel(writer, index=False, header=True, sheet_name=sheet_name, startrow=0 , startcol=0)
        # Get Worksheet Object
        xl_ws = writer.sheets[sheet_name]
        # Turn of Gridlines
        xl_ws.hide_gridlines(2)
        # Write the column data with the defined format.
        for col_num, col_name in enumerate(df.columns.values):
            xl_ws.set_column(col_num, col_num, 15, get_fmt_type(col_name))            
            if col_name in dt_flds:
                for row, val in enumerate(df[col_name]):
                    xl_ws.write_datetime(row+1, col_num, val, date_fmt)

        # Format headers
        for col_num, value in enumerate(df.columns.values):            
            xl_ws.write(0, col_num, value, header_format)
            
        #freezing top row 
        xl_ws.freeze_panes(1, 0) 
            
        # Grab the header rng and try to put a border around the rng
        # hdr_rng = xl_rowcol_to_cell(0, 0) + ':' + xl_rowcol_to_cell(0, len(df.columns)-1)                
        # xl_ws.conditional_format(hdr_rng, {'type' : 'no_blanks', 'format' : border_fmt} )        
        # # Set the border around the left side of the hdr
        # border_fmt.set_left(1)
        # xl_ws.conditional_format(hdr_rng.split(':')[0], {'type' : 'no_blanks', 'format' : border_fmt} )
        # border_fmt.set_left(0)        
        # # Set the border around the right side of the hdr
        # border_fmt.set_right(1)
        # xl_ws.conditional_format(hdr_rng.split(':')[1], {'type' : 'no_blanks', 'format' : border_fmt} )
        # border_fmt.set_right(0)


    write_df_to_sheet_w_fmt(idx_credit_df, idx_credit_df_sheet)
    write_df_to_sheet_w_fmt(inf_summary_view, inf_summary_view_sheet)

    
    # SECTION FOR 1st WORKBOOK
    # xl_ws = xl_wb.get_worksheet_by_name('Idx_Credits')
    # xl_ws = writer.sheets['Idx_Credits']
    # xl_ws.hide_gridlines(2)

    # hdr_rng = xl_rowcol_to_cell(0,0) + ':' + xl_rowcol_to_cell(0, len(idx_credit_df.columns)-1)
    # # Write the column data with the defined format.
    # for col_num, col_name in enumerate(idx_credit_df.columns.values):
    #     xl_ws.set_column(col_num, col_num, 15, get_fmt_type(col_name))
    # # Write the column headers with the defined format.
    # for col_num, value in enumerate(idx_credit_df.columns.values):
    #     xl_ws.write(0, col_num, value, header_format)
    # # Try and put a border around things
    # xl_ws.conditional_format(hdr_rng, {'format' : header_format} )

    # # xl_ws = writer.sheets["Inforce_Summary"]
    # # xl_ws = xl_wb.get_worksheet_by_name('Inforce_Summary')
    # xl_ws = writer.sheets['Inforce_Summary']
    # xl_ws.hide_gridlines(2)

    # Write the column headers with the defined format.
    # for col_num, value in enumerate(inf_summary_view.columns.values):
    #     xl_ws.write(0, col_num, value, header_format)

    

# writer.close()

In [61]:
# hdr_rng = xl_rowcol_to_cell(0,0) + ':' + xl_rowcol_to_cell(0, len(idx_credit_df.columns)-1)
xl_rowcol_to_cell(0,0) + ':' + xl_rowcol_to_cell(0, len(idx_credit_df.columns)-1)

'A1:D1'

In [86]:
df_from_xl_test = pd.read_excel(test_xl_file, sheet_name='Inforce_Summary')

In [87]:
df_from_xl_test

,Inforce_AsOfDt,HedgeDate,PolicyCount,Base_Liab_Ntnl,Adj_Liab_Ntnl
0,2025-01-02,2024-01-02,91481,"58,555,665.68","60,269,646.10"
1,2025-01-02,2024-02-01,92613,"59,738,832.95","61,046,020.25"
2,2025-01-02,2024-03-01,93537,"55,548,864.97","57,006,464.75"
3,2025-01-02,2024-04-01,94712,"55,925,079.27","57,707,624.57"
4,2025-01-02,2024-05-01,95533,"52,870,910.45","56,815,800.07"
...,...,...,...,...,...
77,2024-12-02,2024-07-01,97677,"63,052,655.94","66,348,904.94"
78,2024-12-02,2024-08-01,98210,"58,957,980.57","61,640,511.06"
79,2024-12-02,2024-09-03,99004,"60,257,097.85","61,307,500.49"
80,2024-12-02,2024-10-01,99617,"69,745,247.97","70,717,961.81"


In [59]:
idx_credit_df.dtypes

HedgeDate      object
Indicator      object
PolicyNum       int64
Idx_Credit    float64
dtype: object

In [14]:
test_dirs = ['202405']
test_df = process_subdir_hdgfiles(inforce_fldr, files_to_process=test_dirs)
test_df.to_clipboard()

In [50]:
test_fldr = Path(inforce_fldr)
print(test_fldr)

test_process = [f.name for f in test_fldr.iterdir() if f.is_dir()]
test_process

C:\Users\S0053071\Repos\Orion_Process_Backup\HdgRpts_Results


['202307',
 '202308',
 '202309',
 '202310',
 '202311',
 '202312',
 '202401',
 '202402',
 '202403',
 '202404',
 '202405',
 '202406',
 '202407',
 '202408',
 '202409',
 '202410',
 '202411',
 '202412']

In [65]:
test_df['InforceDt']

1          2024-01-01
2          2024-02-01
3          2024-03-01
4          2024-04-01
5          2024-05-01
              ...    
1249564    2024-12-01
1249565    2024-12-01
1249566    2024-12-01
1249567    2024-12-01
1249568    2024-12-01
Name: InforceDt, Length: 1159235, dtype: object

In [58]:
full_results_df = process_subdir_hdgfiles(inforce_fldr)

In [59]:
full_results_df

,HedgeFile_Dt,Attrib_Type,HedgeDate,CompID,Indicator,Bbg_Idx,Fund_Name,Opt_Type,Strike,Cap/Rate,Budget,PolicyCount,Base_Liab_Ntnl,Notional
0,2023-07-03,New,2023-07-03,1,NASIDX,NDX Index,NASDAQ-100 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.00,0.10,0.05,20.00,"3,287.51","3,188.88"
1,2023-07-03,New,2023-07-03,1,NASIDX,NDX Index,NASDAQ-100 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.00,0.10,0.05,7.00,"5,043.19","4,891.89"
2,2023-07-03,New,2023-07-03,1,NASIDX,NDX Index,NASDAQ-100 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.00,0.10,0.06,"1,607.00","270,547.46","262,431.04"
3,2023-07-03,New,2023-07-03,1,INDEX,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.00,0.07,0.04,373.00,"171,547.68","166,401.25"
4,2023-07-03,New,2023-07-03,1,INDEX,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.00,0.10,0.05,90.00,"509,891.40","494,594.66"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,2024-12-02,Changed,2024-12-02,26,INXFEE,SPX Index,S&P 500 1 Yr Pt-to-Pt Uncapped w Fee,Call,1.09,10.00,0.03,0.00,7.91,6.01
1784,2024-12-02,Changed,2024-12-02,1,INXSPC,SPX Index,S&P 500 1 Yr Specified Rate,Digital,1.00,0.06,0.04,5.00,464.70,441.47
1785,2024-12-02,Changed,2024-12-02,1,INXSPC,SPX Index,S&P 500 1 Yr Specified Rate,Digital,1.00,0.07,0.05,25.00,"8,682.88","8,248.74"
1786,2024-12-02,Changed,2024-12-02,26,INXSPC,SPX Index,S&P 500 1 Yr Specified Rate,Digital,1.00,0.07,0.05,8.00,497.07,377.77


In [61]:
full_results_df.to_clipboard()

In [33]:
# test_sub_dirs = [x[1] for x  in os.walk(results_fldr)]
test_sub_dirs = [x[1] for x in os.walk(inforce_fldr)]
test_sub_dirs = test_sub_dirs[0]

print(test_sub_dirs)

['202307', '202308', '202309', '202310', '202311', '202312', '202401', '202402', '202403', '202404', '202405', '202406', '202407', '202408', '202409', '202410', '202411', '202412']


In [28]:
full_results_df

In [ ]:
test_sub_dir = '202405'
test_dir = os.path.join(inforce_fldr, test_sub_dir)

orig_file = 'iul_liab_summary.csv'
true_file = 'iul_liab_summary_TRUE_UP.csv'

orig_path = os.path.join(test_dir, orig_file)
true_path = os.path.join(test_dir, true_file)

final_cols = ['HedgeFile_Dt', 'Attrib_Type', 'HedgeDate', 'CompID', 'Indicator', 'Bbg_Idx', 'Fund_Name', 'Opt_Type', 'Strike', 'Cap/Rate', 'Budget', 'PolicyCount', 'Base_Liab_Ntnl', 'Notional']
df_cols = ['HedgeDate',	'CompID', 'Indicator', 'Bbg_Idx', 'Fund_Name', 'Opt_Type', 'Strike', 'Cap/Rate', 'Budget', 'PolicyCount', 'Base_Liab_Ntnl', 'Notional']

orig_df = pd.read_csv(orig_path)
orig_df = orig_df[df_cols]

true_df = pd.read_csv(true_path)
true_df = true_df[df_cols]

orig_df['HedgeDate'] = pd.to_datetime(orig_df['HedgeDate']).dt.date
true_df['HedgeDate'] = pd.to_datetime(true_df['HedgeDate']).dt.date
hedge_dt = orig_df['HedgeDate'].max()


In [55]:
test_file = r'G:\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\2_Results\202412\Review Automated VBA\12_2024_HEDGE_ORIG.txt'
hedgefile_colnames = ['CompID', 'PolicyNum', 'Plan', 'IssueDate', 'Indicator', 'Tranx', 'Rev', 'Dept_Desk', 'Entry_Date', 'AsOf_Date', 'Base_Liab_Ntnl', 'Part', 'Cap', 'MGIR_(Cap)', 'Floor', 'Spec_Rate', 'Spread', 'Asset_Charge', 'Multiplier']

test_df = pd.read_csv(test_file, index_col=False, header=None, names=hedgefile_colnames, skipfooter=2, engine='python')

C:\Users\S0053071\AppData\Local\Temp\ipykernel_32400\3241378652.py:4: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  test_df = pd.read_csv(test_file, index_col=False, header=None, names=hedgefile_colnames, skipfooter=2, engine='python')


In [57]:
test_df['Base_Liab_Ntnl'].sum()

np.float64(142757763.13)

In [4]:
from utils.date_utils import get_value_one_row_up, get_full_seriatim_inforce_file, next_bd, prev_bd

In [13]:
inf_obj = inf_dict[list(inf_dict.keys())[0]]

hdgdts_df = inf_obj.assum_dfs['HedgeDates']

In [16]:
print(get_value_one_row_up(hdgdts_df, 'HedgeDate', date(2025, 1, 2)))
print(get_value_one_row_up(hdgdts_df, 'HedgeDate', date(2023, 7, 3)))

2024-12-02
None


In [23]:
processed_inf_fldr = r'C:\Users\S0053071\Repos\Orion_Process_Backup\HdgRpts_Results'
jan_2024_seriatim_orig_df = get_full_seriatim_inforce_file(processed_inf_fldr, date(2025, 1, 2), hdgdts_df)

Creating full seriatim inforce from Orion_IUL_Inforce_wOut_NewCohort.csv and Orig version of HedgeFile!
Saving full seriatim file to C:\Users\S0053071\Repos\Orion_Process_Backup\HdgRpts_Results\202501\Orion_IUL_Inforce_Orig.csv


In [24]:
jan_2024_seriatim_orig_df

,HedgeDate,CompID,PolicyNum,Plan,Indicator,Cap,Ntnl_Mult,Strike,Budget,Base_Liab_Ntnl,Adj_Liab_Ntnl,HedgeRatio,Target_Liab_Ntnl
0,2024-01-02,1,UF002548,1U144600,INDEX,0.10,1.00,1.00,0.05,"1,500.62","1,500.62",0.97,"1,455.60"
1,2024-01-02,1,UF002558,1U144600,INDEX,0.10,1.00,1.00,0.05,"1,556.80","1,556.80",0.97,"1,510.10"
2,2024-01-02,1,UF002808,1U144600,INDEX,0.10,1.00,1.00,0.05,"4,876.74","4,876.74",0.97,"4,730.44"
3,2024-01-02,1,UF002831,1U144600,INDEX,0.10,1.00,1.00,0.05,726.11,726.11,0.97,704.33
4,2024-01-02,1,UF002850,1U144600,INDEX,0.10,1.00,1.00,0.05,541.70,541.70,0.97,525.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259228,2025-01-02,26,UNE06972,1U146900,INXSPC,0.07,1.00,1.00,0.05,16.69,13.35,0.95,12.68
1259229,2025-01-02,26,UNE06983,1U146900,INXSPC,0.07,1.00,1.00,0.05,66.93,53.54,0.95,50.87
1259230,2025-01-02,26,UNS02759,1U146900,INXSPC,0.07,1.00,1.00,0.05,81.14,64.91,0.95,61.67
1259231,2025-01-02,26,UNS02765,1U146900,INXSPC,0.07,1.00,1.00,0.05,"1,610.88","1,288.70",0.95,"1,224.27"


In [25]:
jan_2024_seriatim_true_df = get_full_seriatim_inforce_file(processed_inf_fldr, date(2025, 2, 1), hdgdts_df)

ValDt of 2025-02-01 is prior to the hedge effective date for the month <no new inforce yet>.  Grabbing inforce from previous month!
Attempting to grab inforce received on 2025-01-02
Creating full seriatim inforce from Orion_IUL_Inforce_wOut_NewCohort.csv and TrueUp version of HedgeFile!
Saving full seriatim file to C:\Users\S0053071\Repos\Orion_Process_Backup\HdgRpts_Results\202501\Orion_IUL_Inforce_TrueUp.csv


In [26]:
jan_2024_seriatim_true_df

,HedgeDate,CompID,PolicyNum,Plan,Indicator,Cap,Ntnl_Mult,Strike,Budget,Base_Liab_Ntnl,Adj_Liab_Ntnl,HedgeRatio,Target_Liab_Ntnl
0,2024-01-02,1,UF002548,1U144600,INDEX,0.10,1.00,1.00,0.05,"1,500.62","1,500.62",0.97,"1,455.60"
1,2024-01-02,1,UF002558,1U144600,INDEX,0.10,1.00,1.00,0.05,"1,556.80","1,556.80",0.97,"1,510.10"
2,2024-01-02,1,UF002808,1U144600,INDEX,0.10,1.00,1.00,0.05,"4,876.74","4,876.74",0.97,"4,730.44"
3,2024-01-02,1,UF002831,1U144600,INDEX,0.10,1.00,1.00,0.05,726.11,726.11,0.97,704.33
4,2024-01-02,1,UF002850,1U144600,INDEX,0.10,1.00,1.00,0.05,541.70,541.70,0.97,525.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260803,2025-01-02,26,UNE06972,1U146900,INXSPC,0.07,1.00,1.00,0.05,16.69,13.35,0.95,12.68
1260804,2025-01-02,26,UNE06983,1U146900,INXSPC,0.07,1.00,1.00,0.05,66.93,53.54,0.95,50.87
1260805,2025-01-02,26,UNS02759,1U146900,INXSPC,0.07,1.00,1.00,0.05,81.14,64.91,0.95,61.67
1260806,2025-01-02,26,UNS02765,1U146900,INXSPC,0.07,1.00,1.00,0.05,"1,873.16","1,498.53",0.95,"1,423.60"


In [27]:
get_full_seriatim_inforce_file(processed_inf_fldr, date(2025, 2, 1), hdgdts_df)

ValDt of 2025-02-01 is prior to the hedge effective date for the month <no new inforce yet>.  Grabbing inforce from previous month!
Attempting to grab inforce received on 2025-01-02


,HedgeDate,CompID,PolicyNum,Plan,Indicator,Cap,Ntnl_Mult,Strike,Budget,Base_Liab_Ntnl,Adj_Liab_Ntnl,HedgeRatio,Target_Liab_Ntnl
0,2024-01-02,1,UF002548,1U144600,INDEX,0.10,1.00,1.00,0.05,"1,500.62","1,500.62",0.97,"1,455.60"
1,2024-01-02,1,UF002558,1U144600,INDEX,0.10,1.00,1.00,0.05,"1,556.80","1,556.80",0.97,"1,510.10"
2,2024-01-02,1,UF002808,1U144600,INDEX,0.10,1.00,1.00,0.05,"4,876.74","4,876.74",0.97,"4,730.44"
3,2024-01-02,1,UF002831,1U144600,INDEX,0.10,1.00,1.00,0.05,726.11,726.11,0.97,704.33
4,2024-01-02,1,UF002850,1U144600,INDEX,0.10,1.00,1.00,0.05,541.70,541.70,0.97,525.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260803,2025-01-02,26,UNE06972,1U146900,INXSPC,0.07,1.00,1.00,0.05,16.69,13.35,0.95,12.68
1260804,2025-01-02,26,UNE06983,1U146900,INXSPC,0.07,1.00,1.00,0.05,66.93,53.54,0.95,50.87
1260805,2025-01-02,26,UNS02759,1U146900,INXSPC,0.07,1.00,1.00,0.05,81.14,64.91,0.95,61.67
1260806,2025-01-02,26,UNS02765,1U146900,INXSPC,0.07,1.00,1.00,0.05,"1,873.16","1,498.53",0.95,"1,423.60"


In [6]:
# next_bd, prev_bd
test_dt = date(2025, 2, 3)
print(prev_bd(test_dt))
print(next_bd(test_dt))

2025-01-31
2025-02-04


In [7]:
df_cols = ['HedgeDate', 'CompID', 'PolicyNum', 'Plan', 'Indicator', 'Cap', 'Ntnl_Mult', 'Strike', 'Budget', 'HedgeRatio']
df_cols = df_cols[1:]
df_cols_adj = ['HedgeDate', 'ExpiryDt'] + df_cols
df_cols_adj


['HedgeDate',
 'ExpiryDt',
 'CompID',
 'PolicyNum',
 'Plan',
 'Indicator',
 'Cap',
 'Ntnl_Mult',
 'Strike',
 'Budget',
 'HedgeRatio']